### Gambaran Umum RNNs

RNN adalah jenis neural network yang dapat bekerja pada *sequence* dengan panjang arbitrer. Mereka dapat menganalisis data *time series* seperti harga saham, mengantisipasi lintasan mobil dalam sistem *autonomous driving*, dan berguna untuk aplikasi *Natural Language Processing* (NLP) seperti terjemahan otomatis atau *speech-to-text* karena dapat mengambil kalimat, dokumen, atau sampel audio sebagai *input*.

**Neuron dan Layer Berulang (Recurrent Neurons and Layers):**
Tidak seperti *feedforward neural network* yang aktivasinya mengalir hanya dalam satu arah, RNN memiliki koneksi yang menunjuk ke belakang. Pada setiap langkah waktu ($t$), sebuah neuron berulang menerima *input* $X_{(t)}$ dan *output*-nya sendiri dari langkah waktu sebelumnya, $y_{(t-1)}$. Jika tidak ada *output* sebelumnya pada langkah waktu pertama, biasanya diatur ke 0. Proses ini disebut "membuka" jaringan melalui waktu (unrolling the network through time).

Sebuah *layer* dari neuron berulang dapat dibuat di mana setiap neuron menerima vektor *input* $X_{(t)}$ dan vektor *output* dari langkah waktu sebelumnya, $y_{(t-1)}$. Setiap neuron berulang memiliki dua set bobot: satu untuk *input* $X_{(t)}$ ($W_x$) dan yang lainnya untuk *output* dari langkah waktu sebelumnya, $y_{(t-1)}$ ($W_y$). *Output* dari seluruh *layer* berulang dapat dihitung dengan persamaan berikut:

$Y_{(t)}=\phi({W_{x}}^{T}x_{(t)}+W_{y}^{T}Y_{(t-1)}+b)$

Untuk *mini-batch* penuh, *output* dapat dihitung sebagai:

$Y_{(t)}=\phi(X_{(t)}W_{x}+Y_{(t-1)}W_{y}+b)$

Di sini, $Y_{(t)}$ adalah matriks $m \times n_{neurons}$ yang berisi *output* *layer* pada langkah waktu $t$ untuk setiap *instance* di *mini-batch* ($m$ adalah jumlah *instance*, $n_{neurons}$ adalah jumlah neuron). $X_{(t)}$ adalah matriks $m \times n_{inputs}$ yang berisi *input* untuk semua *instance* ($n_{inputs}$ adalah jumlah *input feature*). $W_x$ dan $W_y$ adalah matriks bobot koneksi untuk *input* dan *output* langkah waktu sebelumnya. $b$ adalah *bias term*.

**Memory Cells:**
Karena *output* neuron berulang pada langkah waktu $t$ adalah fungsi dari semua *input* dari langkah waktu sebelumnya, ia memiliki bentuk memori. Bagian dari *neural network* yang mempertahankan keadaan tertentu di seluruh langkah waktu disebut *memory cell* atau *cell*. Neuron berulang tunggal atau *layer* neuron berulang adalah *cell* dasar yang hanya mampu mempelajari pola pendek (sekitar 10 langkah). Keadaan *cell* pada langkah waktu $t$, $h_{(t)}$ (hidden state), adalah fungsi dari beberapa *input* pada langkah waktu tersebut dan keadaannya pada langkah waktu sebelumnya: $h_{(t)} = f(h_{(t-1)}, x_{(t)})$. *Output*-nya, $Y_{(t)}$, juga merupakan fungsi dari keadaan sebelumnya dan *input* saat ini.

**Input dan Output Sequences:**
RNN dapat secara bersamaan mengambil *sequence* *input* dan menghasilkan *sequence* *output* (sequence-to-sequence). Ini berguna untuk memprediksi *time series*.

Alternatifnya, RNN dapat diberi *sequence* *input* dan mengabaikan semua *output* kecuali yang terakhir (sequence-to-vector). Contohnya adalah memberi jaringan *sequence* kata ulasan film dan mendapatkan skor sentimen.

Sebaliknya, *input* vektor yang sama dapat diberikan berulang kali pada setiap langkah waktu untuk menghasilkan *sequence* *output* (vector-to-sequence). Contohnya adalah menghasilkan *caption* untuk sebuah gambar.

Terakhir, ada model Encoder-Decoder, yang terdiri dari jaringan *sequence-to-vector* (encoder) diikuti oleh jaringan *vector-to-sequence* (decoder). Ini dapat digunakan untuk menerjemahkan kalimat dari satu bahasa ke bahasa lain.

**Pelatihan RNN (Training RNNs):**
Untuk melatih RNN, jaringan dibuka melalui waktu dan kemudian digunakan *backpropagation* reguler. Strategi ini disebut *backpropagation through time* (BPTT). Selama *forward pass*, *output sequence* dievaluasi menggunakan fungsi biaya $C(Y_{(0)}, Y_{(1)}, ..., Y_{(T)})$. Gradien fungsi biaya kemudian disebarkan mundur melalui jaringan yang tidak dilipat, dan parameter model diperbarui. Karena parameter yang sama ($W$ dan $b$) digunakan pada setiap langkah waktu, *backpropagation* akan menjumlahkan semua langkah waktu.

### Peramalan Time Series (Forecasting a Time Series)

*Time series* adalah *sequence* dari satu atau lebih nilai per langkah waktu. Ini bisa univariat (satu nilai per langkah waktu) atau multivariat (beberapa nilai per langkah waktu). Tugas umum adalah memprediksi nilai masa depan (*forecasting*) atau mengisi nilai yang hilang (*imputation*).

**Metrik Baseline (Baseline Metrics):**
Sebelum menggunakan RNN, baik untuk memiliki metrik *baseline*. Pendekatan paling sederhana adalah memprediksi nilai terakhir dalam setiap *series* (*naive forecasting*), yang terkadang sulit dikalahkan. Dalam contoh ini, MSE-nya sekitar 0.020.

Pendekatan lain adalah menggunakan *fully connected network* dengan *Flatten layer* dan *Dense layer*. Model Linear Regression sederhana dapat mencapai MSE sekitar 0.004, jauh lebih baik daripada pendekatan *naive*.

**Mengimplementasikan RNN Sederhana (Implementing a Simple RNN):**
RNN sederhana dapat dibangun dengan `keras.layers.SimpleRNN`. Contoh:
```python
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape=[None, 1])
])
```
*Layer* ini hanya berisi satu *layer* dengan satu neuron. Panjang *sequence* *input* tidak perlu ditentukan karena RNN dapat memproses sejumlah langkah waktu. Secara *default*, *SimpleRNN layer* menggunakan fungsi aktivasi *hyperbolic tangent*. MSE-nya sekitar 0.014, yang lebih baik dari pendekatan *naive* tetapi tidak mengalahkan model linear.

**Deep RNNs:**
Sangat umum untuk menumpuk beberapa *layer* sel untuk membuat *deep RNN*. Ini dilakukan dengan menumpuk *recurrent layer*. Penting untuk mengatur `return_sequences=True` untuk semua *recurrent layer* kecuali yang terakhir (jika hanya *output* terakhir yang penting). Jika tidak, *layer* akan mengeluarkan array 2D, bukan 3D.

Contoh *deep RNN* dengan tiga *SimpleRNN layers*:
```python
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])
```
Model ini mencapai MSE 0.003, mengalahkan model linear.

Untuk meningkatkan fleksibilitas dan kinerja, *layer output* dapat diganti dengan *Dense layer*, dan `return_sequences=True` dihapus dari *recurrent layer* kedua (yang sekarang menjadi yang terakhir):
```python
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])
```
Model ini konvergen lebih cepat dan berkinerja sama baiknya.

**Meramalkan Beberapa Langkah Waktu ke Depan (Forecasting Several Time Steps Ahead):**
Ada dua opsi untuk memprediksi beberapa nilai di masa depan:

1.  **Memprediksi satu langkah waktu pada satu waktu:** Model memprediksi nilai berikutnya, nilai tersebut ditambahkan ke *input*, dan model digunakan lagi. Kesalahan dapat terakumulasi. MSE sekitar 0.029, yang lebih tinggi dari model sebelumnya.
2.  **Melatih RNN untuk memprediksi semua nilai berikutnya sekaligus:** Ini dilakukan dengan mengubah target menjadi vektor yang berisi 10 nilai berikutnya. *Layer output* harus memiliki 10 unit.

    ```python
    model = keras.models.Sequential([
        keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
        keras.layers.SimpleRNN(20),
        keras.layers.Dense(10)
    ])
    ```
    Model ini memiliki MSE sekitar 0.008, jauh lebih baik daripada model linear.

Untuk meningkatkan pelatihan lebih lanjut, model dapat dilatih untuk meramalkan 10 nilai berikutnya pada setiap langkah waktu (sequence-to-sequence). Ini meningkatkan aliran gradien kesalahan, menstabilkan, dan mempercepat pelatihan. Target harus menjadi *sequence* dengan panjang yang sama dengan *input sequence*, berisi vektor 10 dimensi pada setiap langkah.

Untuk mengimplementasikan ini, `return_sequences=True` harus diatur di semua *recurrent layer*, dan *Dense layer* *output* harus diterapkan pada setiap langkah waktu menggunakan `keras.layers.TimeDistributed`.
```python
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])
```
Model ini mencapai MSE validasi sekitar 0.006, 25% lebih baik dari model sebelumnya.

### Menangani Sequence Panjang (Handling Long Sequences)

Melatih RNN pada *sequence* panjang dapat menyebabkan masalah gradien tidak stabil dan *short-term memory* yang terbatas.

**Melawan Masalah Gradien Tidak Stabil (Fighting the Unstable Gradients Problem):**
* Trik yang digunakan dalam *deep net* (inisialisasi parameter yang baik, *optimizer* yang lebih cepat, *dropout*) dapat digunakan untuk RNN.
* Fungsi aktivasi *nonsaturating* (misalnya, ReLU) mungkin tidak membantu dan dapat menyebabkan ketidakstabilan. Fungsi aktivasi *saturating* seperti *hyperbolic tangent* (default) dapat mengurangi risiko ledakan *output* dan gradien.
* *Gradient Clipping* dapat digunakan jika pelatihan tidak stabil.
* *Batch Normalization* tidak seefisien dengan RNN karena tidak dapat digunakan di antara langkah waktu, hanya di antara *recurrent layer*.
* *Layer Normalization* sering bekerja lebih baik dengan RNNs. Ia menormalkan di seluruh dimensi *feature* daripada dimensi *batch*. Ini dapat menghitung statistik dengan cepat pada setiap langkah waktu dan berperilaku sama selama pelatihan dan pengujian.

**Custom Layer Normalization Cell:**
Berikut adalah contoh `LNSimpleRNNCell` yang mengimplementasikan *Layer Normalization* dalam *memory cell* sederhana:
```python
class LNSimpleRNNCell(keras.layers.Layer):
    def __init__(self, units, activation="tanh", **kwargs):
        super().__init__(**kwargs)
        self.state_size = units
        self.output_size = units
        self.simple_rnn_cell = keras.layers.SimpleRNNCell(units, activation=None)
        self.layer_norm = keras.layers.LayerNormalization()
        self.activation = keras.activations.get(activation)

    def call(self, inputs, states):
        outputs, new_states = self.simple_rnn_cell(inputs, states)
        norm_outputs = self.activation(self.layer_norm(outputs))
        return norm_outputs, [norm_outputs]
```
Untuk menggunakan *cell* kustom ini, buat *layer* `keras.layers.RNN` dan berikan *instance* *cell* tersebut:
```python
model = keras.models.Sequential([
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True,
                      input_shape=[None, 1]),
    keras.layers.RNN(LNSimpleRNNCell(20), return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])
```
*Recurrent layer* dan *cell* di Keras juga memiliki *hyperparameter* `dropout` (untuk *input*) dan `recurrent_dropout` (untuk *hidden state*), menghilangkan kebutuhan untuk *cell* kustom untuk *dropout*.

**Mengatasi Masalah Short-Term Memory (Tackling the Short-Term Memory Problem):**
Informasi dapat hilang saat data melintasi RNN, menyebabkan RNN melupakan *input* pertama. Untuk mengatasi ini, diperkenalkan berbagai jenis *cell* dengan memori jangka panjang.

**LSTM Cells:**
*Long Short-Term Memory* (LSTM) *cell* (Hochreiter & Schmidhuber, 1997) berkinerja jauh lebih baik daripada *cell* dasar; pelatihan akan konvergen lebih cepat dan akan mendeteksi ketergantungan jangka panjang dalam data. Di Keras, gunakan *layer* `LSTM`:
```python
model = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.LSTM(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])
```
*Cell* LSTM membagi keadaannya menjadi dua vektor: $h_{(t)}$ (keadaan jangka pendek) dan $c_{(t)}$ (keadaan jangka panjang). Jaringan dapat belajar apa yang harus disimpan, dibuang, dan dibaca dari keadaan jangka panjang. Keadaan jangka panjang $c_{(t-1)}$ melewati *forget gate* dan kemudian menambahkan memori baru melalui operasi penambahan yang dipilih oleh *input gate*. *Output*nya $c_{(t)}$ dikirim langsung keluar.

*Input* saat ini $X_{(t)}$ dan keadaan jangka pendek sebelumnya $h_{(t-1)}$ dimasukkan ke empat *fully connected layer* yang berbeda:
* *Layer* utama mengeluarkan $g_{(t)}$, yang bagian terpentingnya disimpan dalam keadaan jangka panjang.
* Tiga *layer* lainnya adalah *gate controller* menggunakan fungsi aktivasi *logistic* (output 0 hingga 1).
    * *Forget gate* (dikendalikan oleh $f_{(t)}$) mengontrol bagian mana dari keadaan jangka panjang yang harus dihapus.
    * *Input gate* (dikendalikan oleh $i_{(t)}$) mengontrol bagian mana dari $g_{(t)}$ yang harus ditambahkan ke keadaan jangka panjang.
    * *Output gate* (dikendalikan oleh $o_{(t)}$) mengontrol bagian mana dari keadaan jangka panjang yang harus dibaca dan dikeluarkan pada langkah waktu ini sebagai $h_{(t)}$ dan $Y_{(t)}$.

Singkatnya, LSTM dapat belajar mengenali *input* penting, menyimpannya dalam keadaan jangka panjang, mempertahankannya selama dibutuhkan, dan mengekstraknya saat dibutuhkan.

Persamaan LSTM:
$i_{(t)}=\sigma({W_{xi}}^{\top}x_{(t)}+{W_{hi}}^{\top}h_{(t-1)}+b_{i})$
$f_{(t)}=\sigma({W_{xf}}^{\top}x_{(t)}+{W_{hf}}^{\top}h_{(t-1)}+b_{f})$
$o_{(t)}=\sigma({W_{xo}}^{\top}x_{(t)}+W_{ho}{}}^{\top}h_{(t-1)}+b_{o})$
$g_{(t)}=tanh({W_{xg}}^{\top}x_{(t)}+W_{hg}{}^{\top}h_{(t-1)}+b_{g})$
$c_{(t)}=f_{(t)}\otimes c_{(t-1)}+i_{(t)}\otimes g_{(t)}$
$y_{(t)}=h_{(t)}=o_{(t)}\otimes tanh(c_{(t)})$

**Peephole Connections:**
LSTM *cell* dapat ditingkatkan dengan *peephole connection*, di mana *gate controller* juga dapat melihat keadaan jangka panjang ($c_{(t-1)}$ atau $c_{(t)}$). Ini dapat meningkatkan kinerja, tetapi tidak selalu. Keras menyediakan `tf.keras.experimental.PeepholeLSTMCell` untuk ini.

**GRU Cells:**
*Gated Recurrent Unit* (GRU) *cell* (Cho et al., 2014) adalah versi sederhana dari *cell* LSTM yang berkinerja sama baiknya. Penyederhanaan utama adalah:
* Kedua vektor keadaan digabung menjadi satu vektor $h_{(t)}$.
* Satu *gate controller* $Z_{(t)}$ mengontrol *forget gate* dan *input gate*.
* Tidak ada *output gate*; vektor keadaan penuh dikeluarkan pada setiap langkah waktu.
* Ada *gate controller* baru $r_{(t)}$ yang mengontrol bagian mana dari keadaan sebelumnya yang akan ditampilkan ke *layer* utama ($g_{(t)}$).

Persamaan GRU:
$z_{(t)}=\sigma({W_{xz}}^{\top}x_{(t)}+{W_{hz}}^{\top}h_{(t-1)}+b_{z})$
$r_{(t)}=\sigma({W_{xr}}^{\top}x_{(t)}+W_{hr}{}^{\top}h_{(t-1)}+b_{r})$
$g_{(t)}=tanh({W_{xg}}^{\top}x_{(t)}+W_{hg}{}^{\top}(r_{(t)}\otimes h_{(t-1)})+b_{g})$
$h_{(t)}=z_{(t)}\otimes h_{(t-1)}+(1-z_{(t)})\otimes g_{(t)}$

Di Keras, gunakan *layer* `keras.layers.GRU`.

### Menggunakan 1D Convolutional Layers untuk Memproses Sequences (Using 1D Convolutional Layers to Process Sequences)

*Layer convolutional* 1D menggeser beberapa *kernel* di seluruh *sequence*, menghasilkan peta *feature* 1D per *kernel*. Setiap *kernel* belajar mendeteksi pola sekuensial yang sangat pendek. Ini dapat membantu *recurrent layer* mendeteksi pola yang lebih panjang dengan memperpendek *sequence* *input*.

Contoh model dengan *layer convolutional* 1D:
```python
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding="valid",
                        input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])
```
*Layer convolutional* 1D ini *downsamples* *sequence* *input* dengan faktor 2.

**WaveNet:**
WaveNet (Aaron van den Oord et al., 2016) adalah arsitektur yang menumpuk *layer convolutional* 1D, menggandakan *dilation rate* pada setiap *layer*. *Layer* yang lebih rendah belajar pola jangka pendek, sementara *layer* yang lebih tinggi belajar pola jangka panjang. Ini memungkinkan jaringan memproses *sequence* yang sangat besar secara efisien.

Contoh implementasi WaveNet yang disederhanakan:
```python
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=[None, 1]))
for rate in (1, 2, 4, 8) * 2:
    model.add(keras.layers.Conv1D(filters=20, kernel_size=2, padding="causal",
                                  activation="relu", dilation_rate=rate))
model.add(keras.layers.Conv1D(filters=10, kernel_size=1))
```
Model ini menggunakan *padding* "kausal" untuk memastikan *layer convolutional* tidak melihat ke masa depan saat membuat prediksi.